First Data for Passive Data Collection using Smartwatches and GPS from the PREACT Study. 

## Introduction

Treatment personalization is highly discussed to counteract insufficient response rates in psychotherapy. In the quest for criteria allowing informed selection or adaptation, ambulatory assessment data (i.e. EMA, passive sensing)are a key component, as processes happening outside of therapy sessions can be depicted in high temporal and/or spatial resolution.

PREACT is a multicenter prospective-longitudinal study investigating different predictors of non-response (i.e. EEG, fMRI) in around 500 patients undergoing cognitive behavioral therapy for internalizing disorders (https://forschungsgruppe5187.de/de). 

## Methods
Patients can enroll for therapy-accompanying ambulatory assessment. They are provided with a customized study app and a state-of-the-art smartwatch collecting passive data like GPS and heart rate for up to 365 days. In parallel, three 14-day EMA phases (pre-, mid- and post-therapy) cover transdiagnostic (i.e. emotion regulation), contextual and therapy-related aspects.  

Here, we present first results on data compliance and quality for the passive sensing data as well as EMA assessments. The results are based on data that was downloaded on **15.04.2024**.

## Results


In [ ]:
#| echo: false
#| code-fold: true

import os
import glob
import pickle
from IPython.display import Markdown


import pandas as pd
import datetime as dt
from datetime import date, datetime
import numpy as np
from scipy.stats import kruskal, f_oneway


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from config import datapath, proj_sheet
import help_functions 

today = "15042024"

# load data

with open(datapath + f'/ema_data_{today}.pkl', 'rb') as file:
    df_active = pickle.load(file)

with open(datapath + f'/gps_data_{today}.pkl', 'rb') as file:
    df_gps = pickle.load(file)
    
with open(datapath + f'/passive_data_{today}.pkl', 'rb') as file:
    df_passive = pickle.load(file)

with open(datapath + f'/monitoring_data_{today}.pkl', 'rb') as file:
    df_monitoring = pickle.load(file)

In [ ]:
# Extracting month and day of the week
df_monitoring['start_month'] = df_monitoring['ema_start_date'].dt.month
df_monitoring['start_day_of_week'] = df_monitoring['ema_start_date'].dt.dayofweek  # Monday=0, Sunday=6

### Demographics


In [ ]:
# Specify your variables
continuous_vars = ['age', 'bsi_gsi', 'ses', 'costs_ema_burden']
categorical_vars = ['gender_description', 'scid_cv_description', 'ema_smartphone_description', 'employability_description',
                   'prior_treatment_description', 'ema_wear_exp', 'ema_special_event_description', 'partnership']


# Create the table
demographic_table = help_functions.format_demographics(df_monitoring, continuous_vars, categorical_vars)

# Output as Markdown
# Convert to Markdown and display
markdown_output = demographic_table.to_markdown()
display(Markdown(markdown_output))

In [ ]:
# Pie chart for 'scid_cv_description'
scid_counts = df_monitoring['scid_cv_description'].value_counts().reset_index()
scid_counts.columns = ['SCID_CV_Category', 'Counts']  # Renaming columns for clarity in Plotly

fig = px.pie(scid_counts, values='Counts', names='SCID_CV_Category',
             title='Distribution of SCID Primary Categories',
             color_discrete_sequence=px.colors.qualitative.Set3,
             labels={'SCID_CV_Category': 'SCID Category'})  # Labels for hover data
fig.update_layout(showlegend=False)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

### Study status


In [ ]:
#| echo: false
#| code-fold: true

# Creating a subplot layout
fig = make_subplots(rows=1, cols=2, subplot_titles=('Histogram of Study Versions', 'Histogram of Status'))
total_participants = df_monitoring.customer.nunique()

# Histogram for 'study_version'
study_version_data = df_monitoring['study_version'].value_counts().reset_index()
study_version_data.columns = ['Study Version', 'Counts']
fig.add_trace(
    go.Bar(
        x=study_version_data['Study Version'],
        y=study_version_data['Counts'],
        marker=dict(color='lightseagreen'),
        text=study_version_data['Counts'],
        textposition='auto',
    ),
    row=1, col=1
)

# Define active statuses
active_statuses = ['Erhebung_1_aktiv', 'Post_Erhebung_1', 'Post_Erhebung_2', 'Erhebung_2_aktiv']

# Histogram for 'status' with conditional coloring
status_data = df_monitoring['status'].value_counts().reset_index()
status_data.columns = ['Status', 'Counts']

# Generate colors based on condition
colors = ['lightseagreen' if status in active_statuses else 'gray' for status in status_data['Status']]

fig.add_trace(
    go.Bar(
        x=status_data['Status'],
        y=status_data['Counts'],
        marker=dict(color=colors),
        text=status_data['Counts'],
        textposition='auto',
    ),
    row=1, col=2
)

# Update layout settings
fig.update_layout(
    title_text= f"Study version and study status for N= {total_participants} patients",
    showlegend=False,
    height=500, width=1000
)
fig.show()

### GPS


In [ ]:
#| echo: false

df_gps_merged = df_gps.merge(df_monitoring, on = "customer", how="inner")
df_gps_coverage = df_gps_merged.drop_duplicates(subset=["customer"])

In [ ]:
# Assuming df_gps_coverage is your DataFrame
fig = px.violin(df_gps_coverage, 
                y="data_coverage_per", 
                x="ema_smartphone_description", 
                box=True,  # Shows a box plot inside the violin
                title="GPS Coverage per Smartphone")

fig.show()

In [ ]:
# Assuming df_gps_coverage is your DataFrame
fig = px.violin(df_gps_coverage, 
                y="data_coverage_per", 
                x="scid_cv_description", 
                points='all',
                box=True,  # Shows a box plot inside the violin
                title="GPS Coverage per Scid Diagnosis")

fig.show()

### Passive data


In [ ]:
df_pd_merged = df_passive.merge(df_monitoring, on = "customer", how="inner")
df_pd_coverage = df_pd_merged.drop_duplicates(subset=["customer"])

In [ ]:
# Assuming df_gps_coverage is your DataFrame
fig = px.violin(df_pd_coverage, 
                y="data_coverage_per", 
                x="ema_smartphone_description", 
                box=True,  # Shows a box plot inside the violin
                title="Passive Data Coverage per Smartphone")

fig.show()

In [ ]:
# Assuming df_gps_coverage is your DataFrame
fig = px.violin(df_pd_coverage, 
                y="data_coverage_per", 
                x="scid_cv_description", 
                box=True,  # Shows a box plot inside the violin
                points='all',
                title="Passive Data Coverage SCID Diagnosis")

#fig.show()

### EMA data


In [ ]:
# Assuming df_active is your DataFrame and 'nquest_EMA1', 'nquest_EMA2' are your columns
df_active["nquest_EMA1_relative"] = ((df_active["nquest_EMA1"] / 114) * 100).round(0)
df_active["nquest_EMA2_relative"] = ((df_active["nquest_EMA2"] / 114) * 100).round(0)

In [ ]:
#Include only people that have finished the first assessment phase
df_active_coverage = df_active.loc[~df_active.status.isin(['Erhebung_1_aktiv', 'Dropout', 'Pin_missing'])]
df_active_coverage = df_active_coverage.drop_duplicates(subset=["customer"])

In [ ]:
# Calculate the mean and standard deviation for 'nquest_EMA2_relative'
mean_ema2 = df_active_coverage['nquest_EMA2_relative'].mean()
std_dev_ema2 = df_active_coverage['nquest_EMA2_relative'].std()
count_ema2 = df_active_coverage['nquest_EMA2_relative'].count()

# Calculate the mean and standard deviation for 'nquest_EMA1_relative'
mean_ema1 = df_active_coverage['nquest_EMA1_relative'].mean()
std_dev_ema1 = df_active_coverage['nquest_EMA1_relative'].std()
count_ema1 = df_active_coverage['nquest_EMA1_relative'].count()

# Print the results
print(f"Mean for nquest_EMA2_relative: {mean_ema2:.2f} (based on {count_ema2} entries)")
print(f"Standard Deviation for nquest_EMA2_relative: {std_dev_ema2:.2f} (based on {count_ema2} entries)")
print(f"Mean for nquest_EMA1_relative: {mean_ema1:.2f} (based on {count_ema1} entries)")
print(f"Standard Deviation for nquest_EMA1_relative: {std_dev_ema1:.2f} (based on {count_ema1} entries)")

In [ ]:
#| echo: false
#| code-fold: true

study_version_mapping = {
    'Lang': 'Lang',
    'Lang (Wechsel)': 'Lang',
    'Kurz': 'Kurz',
    'Kurz (Wechsel/Abbruch)': 'Kurz'
}
df_active_coverage['Grouped_Study_Status'] = df_active_coverage['study_version'].map(study_version_mapping)

# Isolate the data for the two groups
data_lang = df_active_coverage[df_active_coverage['Grouped_Study_Status'] == 'Lang']['nquest_EMA1_relative']
data_kurz = df_active_coverage[df_active_coverage['Grouped_Study_Status'] == 'Kurz']['nquest_EMA1_relative']

# Perform a t-test between the two groups
stat, p_value = kruskal(data_lang, data_kurz)

# Format the p-value to avoid scientific notation, showing up to 3 decimal places
formatted_p_value = "{:.3f}".format(p_value) if p_value >= 0.001 else "<0.001"

# Create the violin plot using the new grouped category
fig = px.violin(df_active_coverage, 
                y="nquest_EMA1_relative", 
                x="Grouped_Study_Status", 
                box=True,  # Shows a box plot inside the violin
                title="% Completed EMA by Study Status (P-value: {})".format(formatted_p_value))

# Optional: Add custom text annotation if needed to highlight significance visually
fig.add_annotation(x=0.5, xref="paper", yref="paper", y=1.07, text="P-value: {}".format(formatted_p_value),
                   showarrow=False, font=dict(size=12, color="red"))

fig.show()

In [ ]:
#| echo: false
#| code-fold: true

df_active_coverage_plot = df_active_coverage.loc[df_active_coverage.scid_cv_description.isin(['Obsessive-Compulsive Disorder', 'Depressive Disorder', 'Social Anxiety Disorder'])]
groups = df_active_coverage_plot.groupby('scid_cv_description')['nquest_EMA1_relative'].apply(list)

# Performing ANOVA
f_stat, p_value = f_oneway(*groups)

# Print the results
print(f"ANOVA test results — F-statistic: {f_stat}, P-value: {p_value}")

# Create the violin plot
fig = px.violin(df_active_coverage_plot, 
                y="nquest_EMA1_relative", 
                x="scid_cv_description", 
                box=True,  # Shows a box plot inside the violin
                points='all',
                title="% Completed EMA by SCID Diagnosis")

# Displaying the p-value in the plot title if needed
fig.update_layout(title=f"% Completed EMA by SCID Diagnosis (P-value: {p_value:.2})")

fig.show()

### Data coverage across time


In [ ]:
#| echo: false
df_gps_merged_active = df_gps_merged.loc[df_gps_merged.status.isin(['Erhebung_1_aktiv', 'Post_Erhebung_1', 'Post_Erhebung_2','Erhebung_2_aktiv', 'Abgeschlossen'])]
df_gps_merged_active = df_gps_merged_active.loc[df_gps_merged_active.study_version.isin(['Lang', 'Lang(Wechsel)'])]

df_pd_merged_active = df_pd_merged.loc[df_pd_merged.status.isin(['Erhebung_1_aktiv', 'Post_Erhebung_1', 'Post_Erhebung_2','Erhebung_2_aktiv', 'Abgeschlossen'])]
df_pd_merged_active = df_pd_merged_active.loc[df_pd_merged_active.study_version.isin(['Lang', 'Lang(Wechsel)'])]

df_gps_merged_active = df_gps_merged_active.loc[df_gps_merged_active.potential_days_coverage >=60]
df_pd_merged_active = df_pd_merged_active.loc[df_pd_merged_active.potential_days_coverage >=60]

df_ema_active = df_active.loc[~df_active.status.isin(['Erhebung_1_aktiv', 'Dropout', 'Pin_missing'])]
df_ema_active = df_ema_active.loc[df_ema_active.quest_complete_relative1 <=15]

In [ ]:
#| echo: false
#| code-fold: true

# Calculate the total number of unique customers in each DataFrame
total_gps_customers = df_gps_merged_active['customer'].nunique()
total_smartwatch_customers = df_pd_merged_active['customer'].nunique()

# Calculate the number of unique customers per day for GPS and smartwatch data
gps_customer_data = df_gps_merged_active.groupby('relative_day')['customer'].nunique().reset_index()
smartwatch_customer_data = df_pd_merged_active.groupby('relative_day')['customer'].nunique().reset_index()

# Calculate percentages
gps_customer_data['percentage_of_customers'] = (gps_customer_data['customer'] / total_gps_customers) * 100
smartwatch_customer_data['percentage_of_customers'] = (smartwatch_customer_data['customer'] / total_smartwatch_customers) * 100

# Create a DataFrame with all days from 1 to 100
all_days = pd.DataFrame({'relative_day': range(1, 61)})

# Merge and fill missing days with 0 for both datasets
complete_gps_data = all_days.merge(gps_customer_data, on='relative_day', how='left').fillna(0)
complete_smartwatch_data = all_days.merge(smartwatch_customer_data, on='relative_day', how='left').fillna(0)

# Merge both datasets into a single DataFrame for plotting
plot_data = complete_gps_data[['relative_day', 'percentage_of_customers']].rename(columns={'percentage_of_customers': 'gps_percentage'})
plot_data = plot_data.merge(complete_smartwatch_data[['relative_day', 'percentage_of_customers']], on='relative_day', how='left').fillna(0).rename(columns={'percentage_of_customers': 'smartwatch_percentage'})

# Create the line plot
fig = px.line(plot_data,
              x='relative_day',
              y=['gps_percentage', 'smartwatch_percentage'],
              title=f'Coverage of GPS and Smartwatch data. N={str(total_smartwatch_customers)}',
              labels={'value': 'Percentage of available patients', 'variable': 'Data Source', 'relative_day': 'Relative Day'},
              markers=True)

# Add a vertical line at relative day 15
fig.add_vline(x=15, line_width=1, line_dash="dash", line_color="grey")

# Update legend titles
fig.update_layout(legend_title_text='Data Source')
fig.update_traces(mode='lines+markers')

fig.show()

In [ ]:
#| echo: false
#| code-fold: true

df_pd_coverage = df_pd_merged.drop_duplicates(subset=["customer"])

# Calculate the total number of unique customers in each category from the updated DataFrame
total_customers_by_category = df_gps_merged_active.groupby('ema_smartphone_description')['customer'].nunique()

# Calculate the number of unique customers per day for each category from the updated DataFrame
category_customer_data = df_gps_merged_active.groupby(['relative_day', 'ema_smartphone_description'])['customer'].nunique().reset_index()

# Calculate percentages for each category
category_customer_data['percentage_of_customers'] = category_customer_data.apply(
    lambda row: (row['customer'] / total_customers_by_category[row['ema_smartphone_description']]) * 100, axis=1)

# Create a DataFrame with all days from 1 to 100 for merging
all_days = pd.DataFrame({'relative_day': range(1, 61)})

# Merge to ensure every day is represented in each category, fill missing values with 0
plot_data = pd.DataFrame()  # Initialize an empty DataFrame to store the merged data
for category in category_customer_data['ema_smartphone_description'].unique():
    temp_data = category_customer_data[category_customer_data['ema_smartphone_description'] == category]
    merged_data = all_days.merge(temp_data[['relative_day', 'percentage_of_customers']], on='relative_day', how='left').fillna(0)
    merged_data['ema_smartphone_description'] = category
    plot_data = pd.concat([plot_data, merged_data])

# Create the line plot
fig = px.line(plot_data,
              x='relative_day',
              y='percentage_of_customers',
              color='ema_smartphone_description',
              title='GPS coverage by Smartphone Type',
              labels={'percentage_of_customers': '% Patients', 'relative_day': 'Relative Day'},
              markers=True)

# Add a vertical line at relative day 15
fig.add_vline(x=15, line_width=1, line_dash="dash", line_color="grey")

# Update the layout to adjust legend and traces
fig.update_layout(legend_title_text='Smartphone Type')
fig.update_traces(mode='lines+markers')

fig.show()

In [ ]:
# Assuming df_ema_active is your main dataframe and it's already loaded
study_version_mapping = {
    'Lang': 'Lang',
    'Lang (Wechsel)': 'Lang',
    'Kurz': 'Kurz',
    'Kurz (Wechsel/Abbruch)': 'Kurz'
}
df_ema_active['Grouped_Study_Status'] = df_ema_active['study_version'].map(study_version_mapping)

# Calculate the mean of daily_entries_sum for each day and group
mean_daily_entries = df_ema_active.groupby(['quest_complete_relative1', 'Grouped_Study_Status'])['daily_entries_sum'].mean().reset_index()

# Filter the dataframe to include only the relevant x-axis range
mean_daily_entries = mean_daily_entries[(mean_daily_entries['quest_complete_relative1'] >= 0) & (mean_daily_entries['quest_complete_relative1'] <= 15)]

# Create the line plot with separate lines for 'Kurz' and 'Lang'
fig = px.line(mean_daily_entries, 
              x='quest_complete_relative1', 
              y='daily_entries_sum', 
              color='Grouped_Study_Status',  # This separates the lines by group
              title='Mean Number of Completed Daily Surveys Per Day by Study Condition',
              labels={'quest_complete_relative1': 'Day', 'daily_entries_sum': 'Average Daily Entries', 'Grouped_Study_Status': 'Study Condition'})

# Update x-axis to make sure it ranges from 0 to 15
fig.update_layout(
    xaxis=dict(
        range=[0, 15],
        title='Day'
    ),
    yaxis=dict(
        title='Average Daily Entries'
    )
)

# Display the plot
fig.show()

### Subjective Experience (N=68)


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd

# Assuming df_active_coverage is your DataFrame and 'costs_ema_text' is the text column
# Example stop words list
stop_words = set(['die', 'ich', 'und', 'der', 'das', 'ist', 'zu', 'ein', 'in', 'nicht', 'war', 'es', 'mich', 'hat'
                 , 'auf', 'wenn', 'für', 'den', 'mit', 'oder', 'da'])

# Function to remove stop words and handle non-string or NaN entries
def remove_stop_words(text, stop_words):
    if pd.isna(text):
        return ""  # Return an empty string for NaN values
    words = str(text).split()  # Convert to string to handle non-string types
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Apply the function to each text entry
df_active_coverage['filtered_text'] = df_active_coverage['costs_ema_text'].apply(lambda x: remove_stop_words(x, stop_words))

# Combine all text entries into one large string
text = ' '.join(df_active_coverage['filtered_text'])

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Hide the axes
plt.show()

#### 
